In [15]:
# Importación de librerías necesarias

import pandas as pd
import numpy as np

### Acceso a los datos de ventas

Carga el dataset de ventas. ¿Cuántas unidades de cualquier helado se han vendido en total a lo largo del histórico?

In [16]:
df = pd.read_excel('./ventas_helados.xlsx')
df.head()

,fecha,id_producto,ventas_unidades,precio
0,2019-01-07,4758,7953,4.82
1,2019-01-07,17,7052,1.50
2,2019-01-07,9134,6403,2.36
3,2019-01-07,8689,4266,2.47
4,2019-01-07,402,2932,2.89


In [17]:
total_units_sold = df['ventas_unidades'].sum()
print(f'\nSe han vendido un total de: {total_units_sold} helados')


Se han vendido un total de: 3848068 helados


¿Cuál es el identificador del helado con más ventas en euros?

In [18]:
# Agrupamos los datos de ventas por helado y el identificador del helado
ventas_por_helado = df.groupby('id_producto')['ventas_unidades'].sum()
print(ventas_por_helado)

# Encontrar el helado con las ventas más altas
id_helado_max_ventas = ventas_por_helado.idxmax()

print("\nEl ID del helado con más ventas es:", id_helado_max_ventas)

id_producto
17      998119
101      27127
240     202538
402     266723
999      20468
2033    209155
3636     32070
4468     14473
4577    130704
4758    449007
5019     17401
5684     45269
6525     62213
7118      4051
8689    689257
9134    679493
Name: ventas_unidades, dtype: int64

El ID del helado con más ventas es: 17


### Acceso a los datos de publicidad

Lee los datos de publicidad, y transformalos en una serie semanal con la suma de todos los medios.

In [19]:
df = pd.read_excel('./inversiones_helados.xlsx')
df.head()

,fecha,inversion_tv,inversion_radio,inversion_youtube,inversion_facebook
0,2019-01-07,6100,0,6100,0
1,2019-01-14,0,0,0,6530
2,2019-01-21,1120,0,1120,5610
3,2019-01-28,0,0,0,1510
4,2019-02-04,880,880,880,0


In [20]:
# Agrupar por semana y sumar las inversiones de todos los medios
serie_semanal = df.groupby(pd.Grouper(key='fecha')).sum()

# Sumar las inversiones de todos los medios para cada semana
serie_semanal['total_inversion'] = serie_semanal.sum(axis=1)

serie_total_inversion = pd.Series(serie_semanal['total_inversion'], name='total_inversion')

print(serie_total_inversion)

fecha
2019-01-07    12200
2019-01-14     6530
2019-01-21     7850
2019-01-28     1510
2019-02-04     2640
              ...  
2020-11-30    14640
2020-12-07    35160
2020-12-14    79560
2020-12-21     5970
2020-12-28     7680
Name: total_inversion, Length: 104, dtype: int64


¿Cuál es el mes con más inversión total?

In [21]:
# Agrupar por mes y sumar las inversiones de todos los medios
serie_mensual = df.groupby(pd.Grouper(key='fecha', freq='ME')).sum()

# Sumar las inversiones de todos los medios para cada mes
serie_mensual['total_inversion'] = serie_mensual.sum(axis=1)

# Crear la serie mensual con la suma de inversiones totales
serie_total_inversion_mensual = pd.Series(serie_mensual['total_inversion'], name='total_inversion')

print(serie_total_inversion_mensual)

# Encontrar el mes con la mayor inversión total
mes_max_inversion = serie_mensual['total_inversion'].idxmax().strftime('%B de %Y')

print("\nEl mes con la mayor inversión total es:", mes_max_inversion)

fecha
2019-01-31     28090
2019-02-28     29520
2019-03-31     12310
2019-04-30     57890
2019-05-31     26900
2019-06-30     33770
2019-07-31     60080
2019-08-31     80200
2019-09-30     31260
2019-10-31     41680
2019-11-30     30110
2019-12-31     54240
2020-01-31      5210
2020-02-29     69320
2020-03-31     76770
2020-04-30     38780
2020-05-31     25560
2020-06-30     46470
2020-07-31     47810
2020-08-31    141710
2020-09-30     36090
2020-10-31     47610
2020-11-30     31080
2020-12-31    128370
Freq: ME, Name: total_inversion, dtype: int64

El mes con la mayor inversión total es: August de 2020


### Combinando datos

Combina el dataframe de ventas con las descripciones de los productos, y quédate sólo con los productos de helados de tarrina.

In [22]:
df_ventas = pd.read_excel('./ventas_helados.xlsx')
df_productos = pd.read_csv('./nombres_productos.csv')

# Fusionar los DataFrames en función del id_producto
df_combinado = pd.merge(df_ventas, df_productos, on='id_producto', how='inner')

# Filtrar los productos de helados de tarrina
helados_tarrina = df_combinado[df_combinado['nombre'].str.contains('Tarrina')]

helados_tarrina.head()

,fecha,id_producto,ventas_unidades,precio,nombre
5,2019-01-07,2033,1862,2.44,CARTE D' OR Helado de yogur Tarrina 500 gr
8,2019-01-07,240,1015,2.30,Helado Haagen Dazs Cookies Tarrina 400 g
10,2019-01-07,5684,317,3.70,DIA Helado Fresa Tarrina 1kg
11,2019-01-07,101,208,3.93,MAGNUM Helado White Chocolate & Cookies Tarrin...
13,2019-01-07,3636,160,2.60,CARTE D' OR Helado Stracciatella Tarrina 500 gr


Agrega el dataset anterior en una serie semanal que agregue todos los productos mediante la suma de unidades y la media de los precios

In [23]:
serie_semanal_unidades = df_ventas.groupby(pd.Grouper(key='fecha', freq='W'))['ventas_unidades'].sum()
serie_semanal_precios = df_ventas.groupby(pd.Grouper(key='fecha', freq='W'))['precio'].mean()

serie_semanal = pd.Series({'ventas_unidades': serie_semanal_unidades, 'precio_promedio': serie_semanal_precios})

print(serie_semanal)

ventas_unidades    fecha
2019-01-13    35155
2019-01-20    33580
...
precio_promedio    fecha
2019-01-13    2.890000
2019-01-20    2.8...
dtype: object


##### Combina todos los dataframes en uno solo (llamado dataset_conjunto), partiendo del dataset con los datos de helados de tarrina agregados por semana. Utiliza left joins

Crea una variable binaria <i>navidad</i> que valga 1 en las semanas de nochebuena y nochevieja


### Jugando con expresiones regulares

¿Eres capaz de crear una variable de formato (peso en kilos) a partir del nombre del producto, utilizando expresiones regulares?